<a href="https://colab.research.google.com/github/chyan0411/Thesis/blob/master/%E4%B8%8D%E7%94%A8%E7%9C%8Boutput_24_21_21_to_distribution_150_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#这个文件是把21*21 的输出变成 3个label的百分比     (24, 21, 21)  to  (24,3)  最后按照病人 24个变成 (150,3) ： pre，post，fu 
 !pip install import-ipynb
# import import_ipynb

%tensorflow_version 1.x
# !pip install numpy==1.16.1

TensorFlow 1.x selected.


In [ ]:

import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
from keras.models import Model, Sequential
from keras.layers import Conv1D,Flatten,Conv2D, Dropout, MaxPooling2D, Input,Conv3D, MaxPooling3D, Dense, BatchNormalization
from keras import Input,layers
from keras.layers.merge import concatenate 
from google.colab import drive
from keras.optimizers import Adam,RMSprop
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
drive.mount('/content/drive')
# %cp -av fromfolder tofolder
# %cd "/content/drive/My Drive/Newdata/"
# from model_functions import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

Data1 = np.load('/content/drive/My Drive/Newdata/matrix_output_con.npy') 
Data2 = np.load('/content/drive/My Drive/Newdata/matrix_output_pre.npy') 
Data3 = np.load('/content/drive/My Drive/Newdata/matrix_output_post.npy') 
Data4 = np.load('/content/drive/My Drive/Newdata/matrix_output_fu.npy') 

print(Data1.shape)
print(Data3[1])

In [ ]:

def original_441_to_percent(data):
  C = []
  for i in range(24):
    c1 = 0
    c2 = 0
    c3 = 0
    C1 = []
    C2 = []
    C3 = []
    for j in range(21):
      for k in range(21):
        if data[i,j,k] == 0:
          c1+=1
        if data[i,j,k] == 1:
          c2+=1 
        if data[i,j,k] == 2:
          c3+=1
    C1.append(c1/441)
    C2.append(c2/441)
    C3.append(c3/441)
    C.append([C1,C2,C3])
  C = np.array(C)
  C= np.reshape(C,(24,3))
  print(C.shape)
  return C

In [ ]:
con = original_441_to_percent(Data1)
pre = original_441_to_percent(Data2)
post = original_441_to_percent(Data3)
fu = original_441_to_percent(Data4)

(24, 3)
(24, 3)
(24, 3)
(24, 3)


In [ ]:
def make_50_times(data):

  tmp = []
  for i in range(24):
    for j in range(50):
      tmp.append(data[i])
  result = np.array(tmp)
  return result

aa = make_50_times(con)
bb = make_50_times(pre)
cc = make_50_times(post)
dd = make_50_times(fu)
# size变为(1200, 4, 3)
# np.save('/content/drive/My Drive/Newdata/_441to4/_1200_4_con', aa)
# np.save('/content/drive/My Drive/Newdata/_441to4/_1200_4_pre', bb)
# np.save('/content/drive/My Drive/Newdata/_441to4/_1200_4_post', cc)
# np.save('/content/drive/My Drive/Newdata/_441to4/_1200_4_fu', dd) 

In [ ]:
#该每50个就集成一次 按照病人分类， 24个病人 每个病人150个数据（# pre post fu 每50个 ）
for i in range (24):  
  tmp = np.concatenate((bb[i*50:50*(i+1)],cc[i*50:50*(i+1)],dd[i*50:50*(i+1)]),axis=0)
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/percent_output_'+str(i+1),tmp)
  print(i+1)
  print(tmp.shape)  #(150,3)

In [ ]:
patient_order = range(1,25)
order = shuffle(patient_order, random_state = 10)
print(patient_order)
print(order)  
# [14, 18, 20, 5, 2, 22, 19, 15, 23, 17, 24, 13, 11, 6, 9, 7, 1, 3, 8, 12, 21, 10, 16, 4]    random_state =20 
# [6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]    random_state =10

range(1, 25)
[6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]


In [ ]:
# 第一次为 [6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]
# 第二次为 
# order = [6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 17, 18, 1, 16, 5, 10, 19, 11, 21, 24, 9, 20] 
# 第三次为
# order =  [6, 2, 8, 3, 12, 13, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10, 22, 14, 15, 4, 23, 7]
# 第四次为 
order = [22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10, 6, 2, 8, 3, 12, 13]

In [ ]:
tmp = order[0]
A = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(tmp)+ '.npy')
for i in order[1:18]: 
  a = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(i)+ '.npy')
  A = np.concatenate((A,a), axis = 0)
print(A.shape)
# 前十八个 病人数据做训练data(2700, 30, 116, 116)

(2700, 30, 116, 116)


In [ ]:

tmp = order[0]
B = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/percent_output_' + str(tmp)+ '.npy')
for i in order[1:18]: 
  b = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/percent_output_' + str(i)+ '.npy')
  B = np.concatenate((B,b), axis = 0)
print(B.shape)
# 前十八个 病人数据做训练label(2700, 3)

(2700, 3)


In [ ]:
tmp = order[18]
C = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(tmp)+ '.npy')
for i in order[19:]: 
  c = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/115_30_116_116_' + str(i)+ '.npy')
  C = np.concatenate((C,c), axis = 0)
print(C.shape)
# 后六个人 病人数据做测试data(900, 30, 116, 116)

(900, 30, 116, 116)


In [ ]:
tmp = order[18]
D = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/percent_output_' + str(tmp)+ '.npy')
for i in order[19:]: 
  d = np.load('/content/drive/My Drive/Newdata/ordered_by_patient/percent_output_' + str(i)+ '.npy')
  D = np.concatenate((D,d), axis = 0)
print(D.shape)
# 后六个人 病人数据做测试label(900,3)

(900, 3)


In [ ]:
np.save('/content/drive/My Drive/Newdata/ordered_by_patient/按照病人prepost推fu/train_data_4', A)
np.save('/content/drive/My Drive/Newdata/ordered_by_patient/按照病人prepost推fu/train_label_4', B)
np.save('/content/drive/My Drive/Newdata/ordered_by_patient/按照病人prepost推fu/test_data_4', C)
np.save('/content/drive/My Drive/Newdata/ordered_by_patient/按照病人prepost推fu/test_label_4', D)